In [1]:
# coba cari tau RR dan WR (analisis kasus RR & WR)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm
import string
import collections
import numpy as np

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

suffix_df = "/raw/main/results/output/output_df.csv"
suffix_squadid_df = "/resolve/main/results/output/output_df.csv"

In [4]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [5]:
def find_sample_of_rr_and_wr(url):
    
    df = take_dataframe(url)
    
    rr_and_wr_rows = []

    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0]
        nli_validated_answer = df['Prediction Answer'][i]
        
        rr = (gold_answer == qa_generated_answer) and (qa_generated_answer != nli_validated_answer)
        wr = (gold_answer != qa_generated_answer) and (qa_generated_answer != nli_validated_answer)
        
        if wr or rr:
            row = df.iloc[i].copy()
            row['properties'] = 'RR' if rr else 'WR'
            rr_and_wr_rows.append(row)

    rr_and_wr_df = pd.DataFrame(rr_and_wr_rows).reset_index(drop=True)
    
    rr_and_wr_df = rr_and_wr_df.drop(columns=['Properties'])
    rr_and_wr_df = rr_and_wr_df.rename(columns={'properties': 'Properties'})

    return rr_and_wr_df

In [6]:
data = "idkmrc"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
msi = "3"
var = "2"
th = "0.5"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_df}"

rr_wr_df = pd.DataFrame(find_sample_of_rr_and_wr(url))
rr_wr_df

,Prediction Answer,Rec. Pred Answer,Gold Answer,Properties
0,sirkuit terpadu (IC) yang mulai dikembangkan p...,"['', '', 'sirkuit terpadu (IC) yang mulai dike...",,RR
1,1959,"['', '1959', 'GAN yang sedikit ngawur di bidan...",,RR
2,1959,"['', '1959', '1971']",,RR
3,komponen dari sebuah nama pribadi yang berdasa...,"['', 'komponen dari sebuah nama pribadi yang b...",,RR
4,Januari 414,"['', 'Januari 414', '']",,RR
...,...,...,...,...
203,"10 November 1946, ia menjadi orang Oubangui pe...","['', '10 November 1946, ia menjadi orang Ouban...",10 November 1946,WR
204,1960,"['', '1960', '<s>']",,RR
205,<s> Mengapa daerah Nepal dan Bangladesh menyat...,"['', '', '<s> Mengapa daerah Nepal dan Banglad...",,RR
206,"bereproduksi, dan hanya memakan getah pohon hi...","['', 'bereproduksi, dan hanya memakan getah po...",,RR


In [ ]:
#rr_wr_df.to_excel(rr_wr_df.xlsx, index=False)